In [0]:
#all imports
import tensorflow as tf
import torch

In [8]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


In [9]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [10]:
!pip install transformers

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!unzip -P yourpassword -qq '/content/drive/My Drive/GreekData/Greek.zip'

replace Greek/offenseval-greek-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Greek/readme-trainingset-greek.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Greek/offenseval-greek-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [16]:
len(data)

8743

In [17]:
data.to_numpy()[:,2]

array(['OFF', 'NOT', 'NOT', ..., 'NOT', 'NOT', 'NOT'], dtype=object)

In [0]:

dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


In [20]:
pip install emoji --upgrade

     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=8159deb1a4d479f97af0ab76f93cbe742718dd6dad9d91962a4230f9a511bc39
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [21]:
import spacy.cli
spacy.cli.download("el_core_news_md")


✔ Download and installation successful
You can now load the model via spacy.load('el_core_news_md')


In [0]:
#LEMMATIZATION
import string
import spacy
#import el_core_news_sm 
from spacy.tokenizer import Tokenizer
import re
import emoji

nlp =  spacy.load('el_core_news_md')


In [0]:
#Preprocessing inspired from https://stackoverflow.com/questions/4998629/split-string-with-multiple-delimiters-in-python

In [0]:
stop_words=nlp.Defaults.stop_words

In [0]:
txt1=str(x[3].flatten())
delimiters = ";", ".", " ","#"
regexPattern = '|'.join(map(re.escape, delimiters))
a=re.split(regexPattern, txt1)
#a = re.split('|\]|\[|\)|\(|; |,|\*|\n',txt1)
#a=re.split(r'[;,\.\s]\s*', txt1) 

In [46]:
unique_words = dict.fromkeys(a)
print(unique_words)

{"['Με": None, 'Φατσεα': None, 'ξεκινησαμε': None, '': None, "Kokkinopotami']": None}


In [48]:
txt2=' '.join(unique_words)
print(txt2)

['Με Φατσεα ξεκινησαμε  Kokkinopotami']


In [0]:
import emoji
import string
#########DIDNT USE THIS As it didnt improve performance
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        txt1=emoji.demojize(txt)
        #HASHTAG removal 
        #DIDN't WORK SO REMOVED
        delimiters = ";", ".", " ","#",","
        regexPattern = '|'.join(map(re.escape, delimiters))
        a=re.split(regexPattern, txt1)
        unique_words = dict.fromkeys(a)
        txt2=' '.join(unique_words)
        x1=txt2.split(" ")
        x2=[]
        for t in x1:
            z=t
            if z not in string.punctuation  and z not in stop_words:
                x2.append(z)
                   
        
        sentence=' '.join((x2))      
        txt2=' '.join(unique_words)
        ans.append(txt2)
               
    return ans

In [0]:
import spacy.cli
spacy.cli.download("el_core_news_md")

✔ Download and installation successful
You can now load the model via spacy.load('el_core_news_md')


In [0]:
#LEMMATIZATION
import string
import spacy
#import el_core_news_sm 
from spacy.tokenizer import Tokenizer
import re
import preprocessor as p2
import emoji

nlp =  spacy.load('el_core_news_md')
#Preprocessing # to HASHTAG so that spacy can tokenize it properly
p=re.compile(r"(#)",re.UNICODE)
p1=re.compile(r"\.*",re.UNICODE)
##tokenization with NLTK and SPACY DIDNT WORK TOGETHER

In [0]:

          
def preprocess1(arrt):
    doc=[]
    emojis={}
    allTokens =[]
    for txt in arrt:
        k=0
        sentVect=[]
        txt1=emoji.demojize(txt)
        x1=nlp(txt1)
        x2=[]
        for t in x1:
            z=str(t)
            if z not in string.punctuation  and t.is_stop==False:
                x2.append(z)
                   
        
        sentence=' '.join((x2))         
        allTokens.append(sentence)      
    return allTokens


In [0]:
arrt=x[:,0]
allTokens=preprocessinglib(arrt)

In [0]:
preprocessedTweets=allTokens

In [68]:
len(preprocessedTweets)

8743

In [69]:
preprocessedTweets

['@USER Οι μουσουλμάνες που τις βιάζουν έτσι κ αλλιώς  οπότε βολεύει να κυνηγούν εμάς αφήσουν αυτές ήσυχες οι άντρες τους',
 'Η Κάτια προσπαθεί να πείσει οτι δεν είναι ελέφαντας  GynaikaXwrisOnoma',
 "Καλά γιατί λες ότι  είσαι νέος αφού γεννήθηκες το 39 Ναι αλλά στις 29 Φεβρουαρίου κι' έχω γενέθλια κάθε τέσσερα χρόνια οπότε",
 'Με Φατσεα ξεκινησαμε  Kokkinopotami',
 ' gntmgr Κάτια πόσο γλυκιά εμετός',
 'Οι γυναίκες χρειάζονται περισσότερο ύπνο γιατί έχουν πιο σύνθετο εγκέφαλο',
 '@USER Και οι μαλάκες που δικαιολογησαν τον μπάτσο επίσης',
 '@USER Και αντρες και γυναικες αν θελουν να στη κανουν ',
 '@USER Εσύ είσαι η μόνη σωστή εδώ μέσα! Στους άλλους αναφέρομαι που σου κάνουν επίθεση  φασίστες Χρυσαυγιτες',
 '@USER Το ότι αυτό έγινε πολύ μα αργότερα από το Ελληνικό βασίλειο των Μακεδόνων ξέρεις  Επίσης δεν έχει χαλάσει έθνος θες άλλο αν εσύ είσαι ελληνίδα',
 '@USER Αυτή είναι αυστηρώς απαγορευμένη στο σπίτι να ξαναπαίξει  Βαλάντωσαν οι κόρες μου και η γυναίκα Εγώ φυσικά ως σκληρό αντράκι

In [0]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [71]:
y

array([['NOT'],
       ['NOT'],
       ['NOT'],
       ...,
       ['NOT'],
       ['NOT'],
       ['NOT']], dtype=object)

In [73]:
print(preprocessedTweets[0:5])

['Ο Τζόνι λέει ότι η Ρεγγινα έχει εμμονή με τον Παύλο  Ξέχασε να πει και γι αυτό φταίει (η δική του εμμοναρα) Νερτζη poweroflovegr skaipolxeftiles', 'Εντάξει παίρνω πίσω ό τι έγραψα  Με ενημέρωσαν οι φίλοι μου ότι την πήρε η Κάτια Απλα σταμάτησα να βλέπω τη gynaikaxwrisonoma ΜΕ ΚΟΥΡΑΣΕ πολύ μονο ακούω', '@USER απο το επανω τουι  μπορεις να αντιληφθεις προβλημα σε μια ας πουμε Ο Ε που τρεχει για 10 χρονια ή είσαι αυτούς τελικά τους κλέβει ο αλλος και πιανει κορόϊδα ή/και καταστρεφει', '@USER Αεί μωρη καριολα που όποιος δεν γουστάρει τους ισλαμιπιθηκους περιμένεις να σε γαμησουν είναι φασίστας ', "Εγώ όλα τα περιμένω απ' αυτό το μαλακισμένο "]


In [74]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

Using TensorFlow backend.


In [75]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [0]:
yavg=[]

In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]
MAXLENGTH=""
def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
      yavg.append(len(tokenized_sentence_id))
  print(maxlength)
  MAXLENGTH=maxlength
  return MAXLENGTH,yavg
 

In [83]:
#sentence_train, sentence_test, y_train, y_test= train_test_split(preprocessedTweets,y, test_size=0.1, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)
MAXLENGTH,yavg=giveIds(preprocessedTweets,y)
print("Average length is ",sum(yavg)/len(yavg))

155
Average length is  50.65023447329292


In [0]:
##do this at the time of training only 
sentence_train, sentence_test, y_train, y_test = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42) 
#sentence_test,sentence_predict, y_test,y_predict = train_test_split(sentence_test1,y_test1, test_size=0.5, random_state=42)
#x_predict_mask,_,_,_=train_test_split(x_test_mask,y_test, test_size=0.5, random_state=42)

In [87]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
#print(y_predict.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)
yTest=le.fit_transform(y_test.flatten())
print(le.classes_)


(6994, 1)
(1749, 1)
(6994,)
['NOT' 'OFF']
['NOT' 'OFF']


In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [0]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [0]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [0]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [92]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=True)
#for name, param in model.named_parameters():
#	if 'classifier' not in name: # classifier layer
#		param.requires_grad = False
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [94]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek2.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek2.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(optimizer_grouped_parameters,lr=2e-5,eps=1e-8)

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs=4
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

In [100]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])
        #print ("Number of layers:", len(outputs))
        #print ("Number of layers:", len(outputs[0]))

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))
    #torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek2.pth.tar')
    
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  219.    Elapsed time is  25.793530702590942
Batch Completed  100  of  219.    Elapsed time is  51.17350745201111
Batch Completed  150  of  219.    Elapsed time is  76.48994898796082
Batch Completed  200  of  219.    Elapsed time is  101.80988264083862
 The training loss incured is  0.535
  Training one epoch time taken 111.3071870803833
 Validation starts here 
predictionArgmax [0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0]
labelsFlattend [1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0]
labelsFlattend [1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
predictionArgmax [0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0]


In [0]:
torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek2.pth.tar')
